<a href="https://colab.research.google.com/github/kiannylim/SC1015-Data-Science-Project/blob/main/DataCleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import chi2_contingency
from statistics import stdev


In [3]:
data = pd.read_csv("WA_Fn-UseC_-HR-Employee-Attrition.csv")
data

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,No,Travel_Frequently,884,Research & Development,23,2,Medical,1,2061,...,3,80,1,17,3,3,5,2,0,3
1466,39,No,Travel_Rarely,613,Research & Development,6,1,Medical,1,2062,...,1,80,1,9,5,3,7,7,1,7
1467,27,No,Travel_Rarely,155,Research & Development,4,3,Life Sciences,1,2064,...,2,80,1,6,0,3,6,2,0,3
1468,49,No,Travel_Frequently,1023,Sales,2,3,Medical,1,2065,...,4,80,0,17,3,2,9,6,0,8


## Data Cleaning

#### Drop variables that do not have any meaning to attrition (EmployeeCount, EmployeeNumber, Over18, StandardHours)

In [4]:
data = data.drop(columns=["EmployeeCount", "EmployeeNumber", "Over18", "StandardHours"])

#### Drop Data based on bar charts that shows which variables should not be used based on **categorical** data exploratory analysis

In [5]:
data = data.drop(columns=["Department", "Education", "Gender", "JobLevel", "RelationshipSatisfaction", "StockOptionLevel", "WorkLifeBalance"])

#### Drop Data based on bar charts that shows which variables should not be used based on **numerical** data exploratory analysis (store data of the rates for future use in another csv)

In [6]:
rates = data[["DailyRate", "HourlyRate", "MonthlyRate"]]

data = data.drop(columns=["DailyRate", "HourlyRate", "MonthlyRate"])

#### Store income rates in new csv

In [8]:
rates.to_csv('income.csv')

###Chi-Square code

Did a chi-square test on the remaining variables to weed out the independent columns based on the high p-values relative to Attrition.

The null hypothesis states that there is no relationship between the two variables being studied (one variable does not affect the other). **High p-values** indicates probability of an observed result supporting the null hypothesis. A p-value helps you determine the significance of your results in relation to the null hypothesis.

We used a contingency table to represent the frequency of each of our data columns in attrition, in order to pass the values into the chi-square test.
**X axis** as attrition and **y axis** as each of our data columns.

In [ ]:
total_cols = data.columns.tolist() 
total_cols.remove("Attrition")

chi_stat=[]
p_val=[]

for i in total_cols:
    #Generation of our contingency table (or frequency table). With x axis as attrition and y axis as each of our data columns
    observed=pd.crosstab(index=data["Attrition"], columns=data[i])
    chi2, p, dof, ex=chi2_contingency(observed)
    chi_stat.append(chi2)
    p_val.append(p)
    if p >= 0.05:
        print("Attrition and", i ,"are independent variables. p-value =",p )
        print("\n")

chi2 = pd.DataFrame()
chi2["Variable"] = total_cols
chi2["Chi_Statistic"] = chi_stat
chi2["P_value"] = p_val


chi2=chi2[chi2.P_value<0.05]
chi2 = chi2.sort_values("P_value", ascending=True)
display(chi2)

Attrition and DistanceFromHome are independent variables. p-value = 0.09525313501707136


Attrition and MonthlyIncome are independent variables. p-value = 0.7085811366458111


Attrition and PercentSalaryHike are independent variables. p-value = 0.49895522292726846


Attrition and PerformanceRating are independent variables. p-value = 0.9900745465934576


Attrition and YearsSinceLastPromotion are independent variables. p-value = 0.11193387671448679




,Variable,Chi_Statistic,P_value
11,OverTime,87.564294,8.158424e-21
6,JobRole,86.190254,2.752482e-15
8,MaritalStatus,46.163677,9.455511e-11
14,TotalWorkingYears,122.302243,1.586242e-10
0,Age,119.174949,2.575853e-09
19,YearsWithCurrManager,74.617184,3.408337e-09
16,YearsAtCompany,95.388755,2.842980e-07
17,YearsInCurrentRole,64.300944,4.056019e-07
5,JobInvolvement,28.492021,2.863181e-06
1,BusinessTravel,24.182414,5.608614e-06


Drop values that are independent as per the chi-square test

In [9]:
monthlyincome = data["MonthlyIncome"]
data = data.drop(columns=["DistanceFromHome", "MonthlyIncome", "PercentSalaryHike", "PerformanceRating", "YearsSinceLastPromotion"])
data

,Age,Attrition,BusinessTravel,EducationField,EnvironmentSatisfaction,JobInvolvement,JobRole,JobSatisfaction,MaritalStatus,NumCompaniesWorked,OverTime,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsInCurrentRole,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Life Sciences,2,3,Sales Executive,4,Single,8,Yes,8,0,6,4,5
1,49,No,Travel_Frequently,Life Sciences,3,2,Research Scientist,2,Married,1,No,10,3,10,7,7
2,37,Yes,Travel_Rarely,Other,4,2,Laboratory Technician,3,Single,6,Yes,7,3,0,0,0
3,33,No,Travel_Frequently,Life Sciences,4,3,Research Scientist,3,Married,1,Yes,8,3,8,7,0
4,27,No,Travel_Rarely,Medical,1,3,Laboratory Technician,2,Married,9,No,6,3,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,No,Travel_Frequently,Medical,3,4,Laboratory Technician,4,Married,4,No,17,3,5,2,3
1466,39,No,Travel_Rarely,Medical,4,2,Healthcare Representative,1,Married,4,No,9,5,7,7,7
1467,27,No,Travel_Rarely,Life Sciences,2,4,Manufacturing Director,2,Married,1,Yes,6,0,6,2,3
1468,49,No,Travel_Frequently,Medical,4,2,Sales Executive,2,Married,2,No,17,3,9,6,8


In [10]:
data.to_csv('cleaned_dataset.csv')